In [1]:
import numpy as np
import pandas as pd

def safelog2(x):
    if x == 0:
        return(0)
    else:
        return(np.log2(x))

In [2]:
data_set = np.genfromtxt('lab07_data_set.csv', delimiter=',')

N = data_set.shape[0]
D = data_set.shape[1] - 1
print("N = ", N)
print("D = ", D)

X = data_set[:, 0:D]
y = data_set[:, D:(D+1)].astype(int)

print("X.shape = ", X.shape)
print("y.shape = ", y.shape)

K = np.max(y)
print("K = ", K)

# Get first 25 samples of each class, and use them as training data
train_indices = np.concatenate((np.arange(0, 25), 
                                np.arange(50, 75), 
                                np.arange(100, 125)))
test_indices = np.setdiff1d(np.arange(N), train_indices)

X_train = X[train_indices, :]
y_train = y[train_indices, :]
X_test = X[test_indices, :]
y_test = y[test_indices, :]

print("X_train.shape = ", X_train.shape)
print("x_test.shape = ", X_test.shape)

N_train = len(y_train)
N_test = len(y_test)
print("N_train = ", N_train)
print("N_test = ", N_test)

N =  150
D =  4
X.shape =  (150, 4)
y.shape =  (150, 1)
K =  3
X_train.shape =  (75, 4)
x_test.shape =  (75, 4)
N_train =  75
N_test =  75


In [3]:
node_indices = {}
is_terminal = {}
need_split = {}

node_features = {}
node_splits = {}
node_frequencies = {}


node_indices[1] = np.array(range(N_train))
is_terminal[1] = False
need_split[1] = True

In [4]:
# learning algorithm
while True:
    print("iterating...")
    # find nodes that need splitting
    split_nodes = [key for key, value in need_split.items()
                   if value == True]
    # check whether we reach all terminal nodes
    if len(split_nodes) == 0:
        break
    # find best split positions for all nodes
    for split_node in split_nodes:
        data_indices = node_indices[split_node]
        need_split[split_node] = False
        node_frequencies[split_node] = [np.sum(y_train[data_indices] == (c + 1))
                                        for c in range(K)]
        if len(np.unique(y_train[data_indices])) == 1:
            is_terminal[split_node] = True
        else:
            is_terminal[split_node] = False

            best_scores = np.repeat(0.0, D)
            best_splits = np.repeat(0.0, D)
            for d in range(D):
                unique_values = np.sort(np.unique(X_train[data_indices, d]))
                split_positions = (unique_values[1:] + unique_values[:-1]) / 2
                split_scores = np.repeat(0.0, len(split_positions))
                for s in range(len(split_positions)):
                    left_indices = data_indices[X_train[data_indices, d] > \
                                   split_positions[s]]
                    right_indices = data_indices[X_train[data_indices, d] <= \
                                    split_positions[s]]
                    split_scores[s] = -(len(left_indices) / len(data_indices) * \
                                        np.sum([np.mean(y_train[left_indices] == (c + 1)) * \
                                                safelog2(np.mean(y_train[left_indices] == (c + 1)))
                                                for c in range(K)]) + \
                                        len(right_indices) / len(data_indices) * \
                                        np.sum([np.mean(y_train[right_indices] == (c + 1)) * \
                                                safelog2(np.mean(y_train[right_indices] == (c + 1)))
                                                for c in range(K)]))
                best_scores[d] = np.min(split_scores)
                best_splits[d] = split_positions[np.argmin(split_scores)]
            # decide where to split on which feature
            split_d = np.argmin(best_scores)

            node_features[split_node] = split_d
            node_splits[split_node] = best_splits[split_d]
            
            # create left node using the selected split
            left_indices = data_indices[X_train[data_indices, split_d] > \
                           best_splits[split_d]]
            node_indices[2 * split_node] = left_indices
            is_terminal[2 * split_node] = False
            need_split[2 * split_node] = True
      
            # create right node using the selected split
            right_indices = data_indices[X_train[data_indices, split_d] <= \
                            best_splits[split_d]]
            node_indices[2 * split_node + 1] = right_indices
            is_terminal[2 * split_node + 1] = False
            need_split[2 * split_node + 1] = True

iterating...
iterating...
iterating...
iterating...
iterating...
iterating...
